In [ ]:
# 서울시 홈페이지 
# 서울 소식 버튼 클릭
# 게시글 크롤링
# 1페이지 ~5 페이지
# 각각의 페이지 내 게시글 제목 & 게시글을 클릭했을 때 이동하는 페이지 url
# url 이동 후 & 게시글을 업로드한 부서 & 게시날짜
# 어느 부서가 가장 많은 게시물을 업로드했는지 시각화
# 어느 날짜에 게시물들이 많이 업로드 되었는지 시각화

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import time
import pandas as pd
from urllib.parse import urljoin


service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless")
# options.add_argument("--disabe-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 목록별 제목 & 링크값을 수집하는 함수 선언
def collect_list_links_on_current_page() :
    base = driver.current_url # https://www.seoul.go.kr/realmnews/in/list.do
    items = driver.find_elements(By.CSS_SELECTOR, "div.news-lst div.item > a")

    links = []
    for item in items : 
        title = item.find_element(By.CSS_SELECTOR, "em.subject").text.strip()
        href = item.get_attribute("href")   # https://news.seoul.go.kr/gov/archives/574513
        url = urljoin(base, href)
        links.append({"list_title": title, "url": url})
    return links

# 각 페이지당 부서명, 작성날짜 함수 선언
def scape_detail_fields(article_url) :
    driver.get(article_url)

    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#view_top")))

    title = driver.find_element(By.CSS_SELECTOR, "#view_top h3"). text.strip()

    dept_spans = driver.find_elements(By.CSS_SELECTOR, "#view_top dd.dept span")
    # [일자리정책과, 고용훈련팀]

    dept = " - ".join([s.text.strip() for s in dept_spans if s.text.strip()])
    modified_date = driver.find_element(By.CSS_SELECTOR, "#view_top dd.date").text.strip()
    return dept, modified_date
    
try :
    driver.get("https://www.seoul.go.kr/main/index.jsp")
    time.sleep(2)
    
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']"))).click()
    
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.news-lst div.item > a")))

    max_page = 5
    collected_links = []

    for page in range(1, max_page + 1) :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)

        # 함수 호출 지점
        collected_links.extend(collect_list_links_on_current_page())

        print(f"[목록수집] {page}페이지 누적 링크 {len(collected_links)}개")

        if page == max_page : 
            break

        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#paging_main ul.pagination a[data-page='{page+1}']")))
        next_btn.click()

        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.3)

    tmp_df = pd.DataFrame(collected_links).drop_duplicates(subset=["url"]).reset_index(drop=True)
    link_list = tem_df["url"].to_list()

    rows = []
    for i link in enumerate(link_list, start=1) :
        try :
            title, dept, modified_date = scape_detail_fields(link)
            rows.append({
                "title": title,
                "url": link,
                "dept": dept,
                "modified_date": modified_date
            })
            print(f"[상세수집] {i}/{len(link_list)} 완료")

    except Exception as e :
        rows.append({
                "title": "",
                "url": link,
                "dept": "",
                "modified_date": ""
                "error" : str(e)"
            })
            print(f"[상세실패 {i}/{len(link_list)} {link} -> {e}")

        time.sleep(0.3)
    DF = PD.DataFrame(rows)
    filename = "seoul"
    df.to_csv(filename, index=False, encoding)
    
    
finally :
    driver.quit()

[목록수집] 1페이지 누적 링크 10개


JavascriptException: Message: javascript error: Unexpected token ':'
  (Session info: chrome=144.0.7559.60)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xbbccf3
	0xbbcd34
	0x9db1e0
	0x9e1e01
	0x9e4392
	0xa65f2d
	0xa47f0c
	0xa651f3
	0xa47ca6
	0xa19409
	0xa1a1c4
	0xe2a7b4
	0xe25db7
	0xe4379d
	0xbd6ca8
	0xbde87d
	0xbc5708
	0xbc58d2
	0xbaf03a
	0x76bbfcc9
	0x773e82ae
	0x773e827e
